In [1]:

import pandas as pd

from  utils.environment import MyEnvironmentConfig
from utils.df_xtrct_prep import extract_prep_atom_data, extract_prep_episode_data


In [2]:
from match_audit import extract_atom_n_episodes, match_assessments

In [4]:
import mylogger
logger = mylogger.get(__name__)
MyEnvironmentConfig().setup(env='local')

In [5]:
extract_start_date = 20170101 # 20230501 #20220701
extract_end_date = 20230930

atoms, eps  = extract_atom_n_episodes(extract_start_date, extract_end_date)

In [6]:
len(atoms), len(eps)

(8737, 1302)

In [7]:
df_merged_matched =  match_assessments(atoms, eps, matching_ndays_slack=2)

In [8]:
df_merged_matched
# PDC_x is from episode, PDC_y is from atom

,SLK,Program,Staff,AssessmentDate,PDC_x,CommencementDate,EndDate,PDC_y
2,RGNLI170319801,GOLBICE,Anthony.Williams,2022-04-28,Methamphetamine,2022-04-28,2023-01-16,Methamphetamine
3,RGNLI170319801,GOLBICE,Anthony.Williams,2022-06-02,Methamphetamine,2022-04-28,2023-01-16,Methamphetamine
4,RGNLI170319801,GOLBICE,Anthony.Williams,2022-07-14,Methamphetamine,2022-04-28,2023-01-16,Methamphetamine
5,RGNLI170319801,GOLBICE,Anthony.Williams,2022-08-24,Methamphetamine,2022-04-28,2023-01-16,Methamphetamine
6,RGNLI170319801,GOLBICE,Anthony.Williams,2022-10-06,Methamphetamine,2022-04-28,2023-01-16,Methamphetamine
...,...,...,...,...,...,...,...,...
2955,RIBRA260319822,MURMICE,Tina.Raccanello,2023-06-29,Methamphetamine,2023-06-29,NaT,Methamphetamine
2956,RIBRA260319822,MURMICE,Tina.Raccanello,2023-08-01,Methamphetamine,2023-06-29,NaT,Methamphetamine
2957,OYEAL140819561,SAPPHIRE,Liz.Scott,2023-07-19,Ethanol,2023-03-16,NaT,Alcohol
2958,OGNAR070919702,MONPATH,Helen.Waite,2023-07-19,Ethanol,2022-05-27,NaT,Alcohol


In [9]:
eps.head()

,SLK,Program,CommencementDate,EndDate,PDC
0,AL2AZ151120012,MURMHEAD,2022-04-21,2022-07-06,Cannabinoid agonists
1,OSENT160919981,MURMHEAD,2022-05-19,2022-09-08,Methamphetamine
2,OBRRA251020042,MURMHEAD,2022-07-12,2022-08-23,Cannabinoids
3,ET2ON160620011,MURMHEAD,2022-07-27,2022-11-28,Methamphetamine
4,AL2AZ151120012,MURMHEAD,2022-09-13,2022-11-11,Cannabinoid agonists


In [13]:
df_merged_matched.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1977 entries, 2 to 2961
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   SLK               1977 non-null   object        
 1   Program           1977 non-null   object        
 2   Staff             1975 non-null   category      
 3   AssessmentDate    1977 non-null   datetime64[ns]
 4   PDC_x             1977 non-null   category      
 5   CommencementDate  1977 non-null   datetime64[ns]
 6   EndDate           1334 non-null   datetime64[ns]
 7   PDC_y             1977 non-null   object        
dtypes: category(2), datetime64[ns](3), object(3)
memory usage: 118.3+ KB


In [16]:
unique_keys = df_merged_matched[['SLK', 'AssessmentDate', 'PDC_x', 'CommencementDate', 'EndDate', 'Program']].drop_duplicates()
print(unique_keys.shape)

(1977, 6)


In [38]:
assessment_counts = df_merged_matched.groupby(['SLK']).size().reset_index(name='num_assessments')
multiple_assessments = assessment_counts[assessment_counts['num_assessments'] > 1]



In [40]:
multi_atom_slks = multiple_assessments['SLK'].unique()
merged_multiatoms = unique_keys[unique_keys['SLK'].isin(multi_atom_slks)].sort_values(by=['SLK', 'AssessmentDate']) #

In [41]:
merged_multiatoms

,SLK,AssessmentDate,PDC_x,CommencementDate,EndDate,Program
2287,ACARO150719751,2022-09-13,Methamphetamine,2022-09-13,2023-01-09,EUROPATH
2291,ACARO150719751,2023-01-20,Methamphetamine,2023-01-20,2023-05-01,EUROPATH
2295,ACARO150719751,2023-06-08,Methamphetamine,2023-06-08,NaT,EUROPATH
2298,ACARO150719751,2023-08-24,Methamphetamine,2023-06-08,NaT,EUROPATH
1705,ACBOS120219831,2022-03-09,Ethanol,2022-02-25,2022-07-22,GOLBGNRL
...,...,...,...,...,...,...
664,YL2IA310319901,2022-10-20,Ethanol,2022-10-20,2022-11-11,EUROPATH
667,YL2IA310319901,2023-05-04,Ethanol,2023-04-20,NaT,EUROPATH
1810,YNOOB180419881,2022-05-05,"Cannabinoids and Related Drugs, n.f.d.",2022-03-10,2022-11-16,GOLBICE
1811,YNOOB180419881,2022-07-28,"Cannabinoids and Related Drugs, n.f.d.",2022-03-10,2022-11-16,GOLBICE


In [45]:
# Step 2: Identify inconsistent PDC_x within each group

merged_multiatoms.sort_values(by=['SLK', 'Program', 'CommencementDate', 'EndDate', 'AssessmentDate'], inplace=True)
groups = merged_multiatoms.groupby(['SLK', 'Program', 'CommencementDate', 'EndDate'])
to_delete_inconsistent_PDC = []
for name, group in groups:
    unique_PDCs = group['PDC_x'].nunique()
    if unique_PDCs > 1:
        to_delete_inconsistent_PDC.extend(group.index.tolist())
# Step 3: Identify assessments within the range of a different episode with different PDC_x
to_delete_crossing_episodes = []

for i, row in merged_multiatoms.iterrows():
    mask = (
        (merged_multiatoms['SLK'] == row['SLK']) &
        (merged_multiatoms['Program'] == row['Program']) &
        (merged_multiatoms['PDC_x'] != row['PDC_x']) &
        (merged_multiatoms['CommencementDate'] <= row['AssessmentDate']) &
        (merged_multiatoms['EndDate'] >= row['AssessmentDate'])
    )
    
    if merged_multiatoms[mask].shape[0] > 0:
        to_delete_crossing_episodes.append(i)

# Combine indices for deletion
to_delete = list(set(to_delete_inconsistent_PDC + to_delete_crossing_episodes))

# Delete the rows
df_clean = merged_multiatoms.drop(index=to_delete)



NameError: name 'df' is not defined

In [47]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1442 entries, 2287 to 1812
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   SLK               1442 non-null   object        
 1   AssessmentDate    1442 non-null   datetime64[ns]
 2   PDC_x             1442 non-null   category      
 3   CommencementDate  1442 non-null   datetime64[ns]
 4   EndDate           870 non-null    datetime64[ns]
 5   Program           1442 non-null   object        
dtypes: category(1), datetime64[ns](3), object(2)
memory usage: 70.4+ KB


In [55]:
# df_clean.PDC_x.unique()

['Methamphetamine', 'Ethanol', 'Cannabinoids and Related Drugs, n.f.d.', 'Cocaine', 'Amphetamines, n.f.d.', ..., 'Amphetamines, n.f.d', 'Amphetamine', 'Pharmaceutical Opioids, n.f.d.', 'Benzodiazepines, n.f.d.', 'Heroin']
Length: 13
Categories (46, object): ['Amphetamines, n.f.d.', 'Ethanol', 'Cannabinoids and Related Drugs, n.f.d.', 'Amphetamines, n.f.d', ..., 'Codeine', 'Not Stated', 'Benzodiazepines, n.e.c.', 'Acetone ']

In [57]:
min_max_assessments = df_clean.groupby(['SLK',  'CommencementDate',  'PDC_x']).agg(
    {
        'AssessmentDate': ['min', 'max']
    }
).reset_index()

In [59]:
# Flatten the MultiIndex

min_max_assessments.columns = ['_'.join(col).rstrip('_') for col in min_max_assessments.columns.values]
# Filter out rows where both min and max of AssessmentDate are NaT

min_max_assessments_filtered = min_max_assessments.dropna(subset=['AssessmentDate_min', 'AssessmentDate_max'], how='all')


In [60]:
min_max_assessments_filtered

,SLK,CommencementDate,PDC_x,AssessmentDate_min,AssessmentDate_max
8055,ACARO150719751,2022-09-13,Methamphetamine,2022-09-13,2022-09-13
10815,ACARO150719751,2023-01-20,Methamphetamine,2023-01-20,2023-01-20
13943,ACARO150719751,2023-06-08,Methamphetamine,2023-06-08,2023-08-24
17987,ACBOS120219831,2022-02-25,Ethanol,2022-03-09,2022-06-24
39239,ACEUR140319611,2022-12-22,Ethanol,2022-12-22,2023-05-25
...,...,...,...,...,...
6280795,YE2AR171019842,2023-03-23,Ethanol,2023-03-23,2023-03-23
6287331,YGAUK280819881,2022-04-06,Methamphetamine,2022-06-24,2023-07-27
6306417,YL2IA310319901,2022-10-20,Ethanol,2022-10-20,2022-10-20
6310419,YL2IA310319901,2023-04-20,Ethanol,2023-05-04,2023-05-04


In [61]:
min_max_assessments_filtered[min_max_assessments_filtered['SLK'] =='ILIAY060820032' ]

,SLK,CommencementDate,PDC_x,AssessmentDate_min,AssessmentDate_max
2997592,ILIAY060820032,2022-08-24,"Cannabinoids and Related Drugs, n.f.d.",2022-08-24,2022-12-21


In [62]:
merged_multiatoms[ merged_multiatoms['SLK'] =='ILIAY060820032' ]

,SLK,AssessmentDate,PDC_x,CommencementDate,EndDate,Program
1100,ILIAY060820032,2021-08-17,MDMA/Ecstasy,2021-08-17,2022-08-23,GOLBICE
1102,ILIAY060820032,2021-09-28,MDMA/Ecstasy,2021-08-17,2022-08-23,GOLBICE
1104,ILIAY060820032,2021-11-11,MDMA/Ecstasy,2021-08-17,2022-08-23,GOLBICE
1106,ILIAY060820032,2021-12-22,MDMA/Ecstasy,2021-08-17,2022-08-23,GOLBICE
1108,ILIAY060820032,2022-02-15,MDMA/Ecstasy,2021-08-17,2022-08-23,GOLBICE
1110,ILIAY060820032,2022-03-30,MDMA/Ecstasy,2021-08-17,2022-08-23,GOLBICE
1112,ILIAY060820032,2022-05-26,MDMA/Ecstasy,2021-08-17,2022-08-23,GOLBICE
1114,ILIAY060820032,2022-07-12,MDMA/Ecstasy,2021-08-17,2022-08-23,GOLBICE
1116,ILIAY060820032,2022-08-24,"Cannabinoids and Related Drugs, n.f.d.",2021-08-17,2022-08-23,GOLBICE
1117,ILIAY060820032,2022-08-24,"Cannabinoids and Related Drugs, n.f.d.",2022-08-24,2023-02-09,GOLBICE


In [33]:
atom_counts = df_merged_matched.groupby(['SLK','AssessmentDate']).size().reset_index(name='num_atoms')



In [34]:
gt_atoms = atom_counts[atom_counts['num_atoms'] > 1]
gt_atoms

,SLK,AssessmentDate,num_atoms
950,ILIAY060820032,2022-08-24,2
1035,ITLAT020619801,2022-09-15,2


In [27]:
df_merged_matched[ df_merged_matched['SLK'] =='ILIAY060820032']

,SLK,Program,Staff,AssessmentDate,PDC_x,CommencementDate,EndDate,PDC_y
1100,ILIAY060820032,GOLBICE,Anthony.Williams,2021-08-17,MDMA/Ecstasy,2021-08-17,2022-08-23,MDMA/Ecstasy
1102,ILIAY060820032,GOLBICE,Anthony.Williams,2021-09-28,MDMA/Ecstasy,2021-08-17,2022-08-23,MDMA/Ecstasy
1104,ILIAY060820032,GOLBICE,Anthony.Williams,2021-11-11,MDMA/Ecstasy,2021-08-17,2022-08-23,MDMA/Ecstasy
1106,ILIAY060820032,GOLBICE,Anthony.Williams,2021-12-22,MDMA/Ecstasy,2021-08-17,2022-08-23,MDMA/Ecstasy
1108,ILIAY060820032,GOLBICE,Anthony.Williams,2022-02-15,MDMA/Ecstasy,2021-08-17,2022-08-23,MDMA/Ecstasy
1110,ILIAY060820032,GOLBICE,Anthony.Williams,2022-03-30,MDMA/Ecstasy,2021-08-17,2022-08-23,MDMA/Ecstasy
1112,ILIAY060820032,GOLBICE,Anthony.Williams,2022-05-26,MDMA/Ecstasy,2021-08-17,2022-08-23,MDMA/Ecstasy
1114,ILIAY060820032,GOLBICE,Anthony.Williams,2022-07-12,MDMA/Ecstasy,2021-08-17,2022-08-23,MDMA/Ecstasy
1116,ILIAY060820032,GOLBICE,Anthony.Williams,2022-08-24,"Cannabinoids and Related Drugs, n.f.d.",2021-08-17,2022-08-23,MDMA/Ecstasy
1117,ILIAY060820032,GOLBICE,Anthony.Williams,2022-08-24,"Cannabinoids and Related Drugs, n.f.d.",2022-08-24,2023-02-09,Cannabinoids


In [64]:
# SLKs with more than one PDC_x
pdc_counts  = df_merged_matched.groupby(['SLK'])['PDC_x'].nunique().reset_index(name='num_unique_pdc')
multiple_pdc = pdc_counts[pdc_counts['num_unique_pdc'] > 1]



In [66]:
len(multiple_pdc)

30